In [1]:
import awswrangler as wr
import pandas as pd

from data_sci_toolkit.aws_tools import permission_tools 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

/opt/miniconda3/envs/stellar/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
SESSION = permission_tools.get_aws_session("Dev-DataScienceAdmin")

Authorized as AROAQBUNWK7XN5DTYWPLJ:rkazmerik@stellaralgo.com


In [3]:
DATABASE = "integrations_milb_aviators"
S3_BUCKET = "s3://dev-athena-testing-us-east-1"

In [11]:
df2 = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    ctas_approach= False,
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """
        WITH customers as (
            select * 
            from integrations_milb_aviators.customers c
            CROSS JOIN UNNEST(account_ids['ticketing'], account_ids['retail']) as t(ticket_ids, retail_ids)
        ),
        ticketing AS (
            select *
            from integrations_milb_aviators.tickets t
            CROSS JOIN UNNEST(resales, forwards, scans) as t(resale_data, forward_data, scan_data)
            where returned_at IS NULL
        )
        
        select * 
        from ticketing t
        INNER JOIN integrations_milb_aviators.ticketing_events e
        on t.event_id = e.id
        INNER JOIN customers c
        on c.ticket_ids = t.account_id limit 10 
    """
)
df2.shape

(10, 75)

In [12]:
df2

,source_system,source_system_group,reported_revenue,product,client_property,seat_id,section_name,row_name,seat_num,season_id,event_id,event_name,plan_event_id,ticket_type,comp_code,promo_code,account_id,default_scv_id,price_code,price,account_rep_id,account_rep_name,purchased_at,returned_at,updated_at,forwards,resales,scans,resale_data,forward_data,scan_data,source_system.1,source_system_group.1,id,is_renewal,client_property.1,name,full_name,venue,team,start_date,end_date,day_of_week,time,time_of_day,event_type,season_id.1,season,season_year,game_number,min_events,total_events,fse,has_payment_plan,major_category,minor_category,id.1,account_ids,email,first_name,last_name,address,city,state,country,zip,phone,company_name,account_type,account_rep_name.1,demographics,history,venues,ticket_ids,retail_ids
0,ticketmaster,ticketing,75.0,Individual,milb_aviators,E88CE42E73F00EBC31B6323C7F65166D,DOP105,A,5,63,2976,EV200307,<NA>,Dugout Prime,0,<NA>,1103686,4fa2215df13a1625748b3ab54b04bd18,DS,75.0,16,Erik Eisenberg,2020-01-07 12:20:45.041,NaT,NaT,"[{source_account_id=1103686, account_id=111914...",NaN,"[{account_id=1119149, scanned_at=null, scanned...",NaN,"{source_account_id=1103686, account_id=1119149...","{account_id=1119149, scanned_at=null, scanned_...",ticketmaster,ticketing,2976,True,other,EV200307,Cincinnati Vs Chicago 3/7,Las Vegas Ballpark,Cincinnati vs Chicago,2020-03-07,2020-03-07,SAT,13:05:00,Afternoon,Single Event,63,2020 Big League Weekend,2019,3,1,1,0.0,False,Sports,Mlb Pro Baseball,35eadbe3eb003b2baff97a7446b7d620,"{ticketing=[1103686], retail=[2568364916818]}",tim@dynamicpricingpartners.com,Tim,Devlin,1111 44th Dr Ste S,Long Island City,New York,United States,11101,+1 (631) 291-3578,Dynamic Pricing Partners,Season Ticket,Erik Eisenberg,NaN,NaN,"[{name=Las Vegas Ballpark, distance=3607.04605...",1103686,2568364916818
1,ticketmaster,ticketing,65.0,Individual,milb_aviators,4C090F4227486EB9709BD9EB2B368E67,DOB105,F,3,63,2974,EV200229,<NA>,Dugout Box,0,<NA>,1723891,9ed0fbedda75d5090fb216c8bd0d84f9,ES,65.0,28,Bryan Frey,2019-12-14 10:11:27.357,NaT,NaT,"[{source_account_id=1723891, account_id=177675...",NaN,"[{account_id=1776758, scanned_at=null, scanned...",NaN,"{source_account_id=1723891, account_id=1776758...","{account_id=1776758, scanned_at=null, scanned_...",ticketmaster,ticketing,2974,True,other,EV200229,Cleveland Vs Oakland 2/29,Las Vegas Ballpark,Cleveland vs Oakland,2020-02-29,2020-02-29,SAT,13:05:00,Afternoon,Single Event,63,2020 Big League Weekend,2019,1,1,1,0.0,False,Sports,Mlb Pro Baseball,6ecd5912eccf480d76c7afff5035a980,"{ticketing=[1723891, 1848027, 2368354], retail...",steven.polis6@gmail.com,Steven,Polis,8 Eddlewood Pl,Mount Laurel,New Jersey,United States,08054,+1 (856) 316-6876,Premium Live Entertainment,Personal,Michelle Taggart,NaN,NaN,"[{name=Las Vegas Ballpark, distance=3534.44853...",1723891,2967484039250
2,ticketmaster,ticketing,0.0,Comp,milb_aviators,AC7F81923D07AE728BDC5F54349558E2,DOB105,N,4,63,2977,EV200308,<NA>,<NA>,1,<NA>,280545,f55cb1d5744372e31902bd10ca4f60cd,EC,0.0,16,Erik Eisenberg,2019-12-18 15:55:53.370,NaT,NaT,NaN,NaN,"[{account_id=280545, scanned_at=null, scanned_...",NaN,NaN,"{account_id=280545, scanned_at=null, scanned_t...",ticketmaster,ticketing,2977,True,other,EV200308,Cincinnati Vs Chicago 3/8,Las Vegas Ballpark,Cincinnati vs Chicago,2020-03-08,2020-03-08,SUN,13:05:00,Afternoon,Single Event,63,2020 Big League Weekend,2019,4,1,1,0.0,False,Sports,Mlb Pro Baseball,4807bad66a28086965161b7c7327993b,"{ticketing=[280545, 1896201], retail=[57491993...",ticketpartners@lvcva.com,Lisa,Motley,3150 Paradise Rd.,Las Vegas,Nevada,United States,89109,<NA>,Lvcva,Personal,Michelle Taggart,NaN,NaN,"[{name=Las Vegas Ballpark, distance=16.6819612...",280545,5749199339602
3,ticketmaster,ticketing,55.0,Individual,milb_aviators,84002FEC15DDDAFCA6E894984531A27B,BERMG0,GA0,42,63,2977,EV200308,<NA>,Adult,0,<NA>,1780502,f882e8532c9eb579bb3250340e6e2048,I*,55.0,14,Michelle Taggart,2020-01-19 16:27:06.810,NaT,N